In [27]:
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated

In [29]:
imgs = MNIST.images()   #loading images
imgs[1]

In [32]:
X = hcat(float.(reshape.(imgs, :))...) |> gpu        # converting images to vector format using gpu support and concatenating result of each image

784×60000 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [4]:
labels = MNIST.labels()        # loading labels of images and convertsing to one-hot encoding
Y = onehotbatch(labels, 0:9) |> gpu

10×60000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 false   true  false  false  false  …  false  false  false  false  false
 false  false  false   true  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false   true  false  false  false
 false  false   true  false  false     false  false  false  false  false
  true  false  false  false  false  …  false  false   true  false  false
 false  false  false  false  false     false  false  false   true  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false      true  false  false  false   true
 false  false  false  false   true     false  false  false  false  false

In [26]:
m = Chain(                           # creating a simple MLP model
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax) |> gpu

loss(x, y) = crossentropy(m(x), y)        # defining loss function where m(x) will be predicted and y the target

accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))   # defining validation metric

accuracy (generic function with 1 method)

In [28]:
dataset = repeated((X, Y), 200)    # here the way of specifying epochs is to repeat data i.e 200 times
evalcb = () -> @show(loss(X, Y))
opt = ADAM(params(m))                      #setting optimizer

Flux.train!(loss, dataset, opt, cb = throttle(evalcb, 10))   # cb does some operation after every 10 seconds

accuracy(X, Y)

loss(X, Y) = 2.3863134087456896 (tracked)
loss(X, Y) = 1.5902715570841042 (tracked)
loss(X, Y) = 1.0227195329263457 (tracked)
loss(X, Y) = 0.704610835948741 (tracked)
loss(X, Y) = 0.5567989282723075 (tracked)
loss(X, Y) = 0.46971012029187015 (tracked)
loss(X, Y) = 0.418928345119825 (tracked)
loss(X, Y) = 0.3812787977256498 (tracked)
loss(X, Y) = 0.3566023468685513 (tracked)
loss(X, Y) = 0.33568823008272086 (tracked)
loss(X, Y) = 0.3196388247363892 (tracked)
loss(X, Y) = 0.3050524903338718 (tracked)
loss(X, Y) = 0.2932783920949663 (tracked)
loss(X, Y) = 0.2829013999081407 (tracked)


0.9236833333333333

In [30]:
# Test set accuracy
tX = hcat(float.(reshape.(MNIST.images(:test), :))...) |> gpu
tY = onehotbatch(MNIST.labels(:test), 0:9) |> gpu

accuracy(tX, tY)

0.9248